<a href="https://colab.research.google.com/github/Raiyas91/Credit-card-Fraud-Detection/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies

In [44]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec

Load the dataset

In [6]:
credit_card_data = pd.read_csv("/content/creditcard.csv")

Print first five rows.

In [7]:
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [8]:
#dataset information
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162557 entries, 0 to 162556
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    162557 non-null  float64
 1   V1      162557 non-null  float64
 2   V2      162557 non-null  float64
 3   V3      162557 non-null  float64
 4   V4      162557 non-null  float64
 5   V5      162557 non-null  float64
 6   V6      162557 non-null  float64
 7   V7      162557 non-null  float64
 8   V8      162557 non-null  float64
 9   V9      162557 non-null  float64
 10  V10     162557 non-null  float64
 11  V11     162557 non-null  float64
 12  V12     162557 non-null  float64
 13  V13     162557 non-null  float64
 14  V14     162557 non-null  float64
 15  V15     162557 non-null  float64
 16  V16     162557 non-null  float64
 17  V17     162557 non-null  float64
 18  V18     162557 non-null  float64
 19  V19     162557 non-null  float64
 20  V20     162557 non-null  float64
 21  V21     16

In [9]:
#checking the missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       1
Amount    1
Class     1
dtype: int64

In [10]:
# distribution of legit transaction and fraudulent transaction
credit_card_data['Class'].value_counts()

0.0    162200
1.0       356
Name: Class, dtype: int64

In [11]:
#separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [12]:
print(legit.shape)
print(fraud.shape)

(162200, 31)
(356, 31)


In [13]:
1 # Statistical measures of the data
legit.Amount.describe()

count    162200.000000
mean         86.929842
std         244.400180
min           0.000000
25%           5.380000
50%          21.350000
75%          76.070000
max       19656.530000
Name: Amount, dtype: float64

In [14]:
fraud.Amount.describe()

count     356.000000
mean      112.005000
std       228.517058
min         0.000000
25%         1.000000
50%         9.905000
75%       102.125000
max      1809.680000
Name: Amount, dtype: float64

In [15]:
# Compare the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,58176.368921,-0.185441,0.042134,0.569680,0.124500,-0.191311,0.067597,-0.076600,0.038895,0.027550,...,0.032755,-0.036010,-0.096158,-0.02534,0.010510,0.103687,0.014082,0.001871,0.002442,86.929842
1.0,55968.873596,-6.057284,4.419181,-8.006879,4.902106,-4.402531,-1.445960,-7.057336,0.778068,-2.884994,...,0.413879,0.767732,-0.017997,-0.04505,-0.073773,0.080805,0.036841,0.181541,0.044143,112.005000


Under Sampling

Build a sample dataset containing similar distribution of normal transaction and fraudulent transaction

Number of fraudulent transactions -> 492

In [16]:
legit_sample = legit.sample(n=492)

Concatenating two DataFrames

In [17]:
new_dataset = pd.concat([legit_sample, fraud], axis = 0)

In [18]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
71582,54349.0,1.151517,0.022631,0.102991,1.134340,0.130690,0.513680,-0.118203,0.241516,0.258076,...,-0.103468,-0.173367,-0.153331,-0.679949,0.681152,-0.284580,0.025597,-0.002064,17.68,0.0
4303,3758.0,-0.812809,-0.019916,2.123482,-1.524092,-0.078559,-0.904809,-0.033882,-0.063047,-0.119061,...,-0.075700,-0.279277,0.103711,0.314355,-0.168617,-0.673335,0.036935,0.101520,25.00,0.0
123215,76848.0,1.335609,0.003761,-0.375240,-1.128282,0.160692,-0.505427,0.160530,-0.210449,0.919300,...,0.027873,0.326596,-0.230687,-0.711931,0.881474,-0.523183,0.063024,0.008331,1.00,0.0
11718,20031.0,0.928848,0.170297,1.276773,2.714846,0.247682,2.136157,-0.793681,0.595445,1.229087,...,0.047688,0.700091,0.032208,-0.981526,0.199405,0.237053,0.073797,0.009761,16.30,0.0
17849,28974.0,-0.576487,1.022091,1.628924,-0.374750,0.225452,-1.038670,0.875820,-0.121828,-0.594405,...,-0.245282,-0.828794,-0.134234,0.337449,0.310325,0.011952,-0.076412,-0.014384,9.99,0.0


In [19]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
156990,109298.0,-1.000611,3.346850,-5.534491,6.835802,-0.299803,0.095951,-2.440419,1.286301,-2.766438,...,0.439757,-0.694099,0.299660,-0.657601,0.101648,0.430457,0.824685,0.326952,186.13,1.0
157585,110087.0,1.934946,0.650678,-0.286957,3.987828,0.316052,-0.099449,-0.021483,-0.172327,0.508730,...,-0.173602,-0.190974,0.219976,-0.216597,-0.136692,-0.129954,-0.050077,-0.051082,1.00,1.0
157868,110547.0,-1.532810,2.232752,-5.923100,3.386708,-0.153443,-1.419748,-3.878576,1.444656,-1.465542,...,0.632505,-0.070838,-0.490291,-0.359983,0.050678,1.095671,0.471741,-0.106667,0.76,1.0
157871,110552.0,-2.450367,2.107729,-5.140663,1.411304,-1.690780,-0.736427,-3.657946,1.944906,-0.788388,...,0.800538,0.364617,0.233608,-0.282078,-0.320311,0.492920,0.359976,-0.115471,80.22,1.0
157918,110617.0,-1.101035,-1.674928,-0.573388,5.617556,0.765556,0.440607,1.934740,-1.019788,-0.193244,...,-0.412526,-0.208823,0.344833,1.091435,-0.686513,0.075809,0.033865,-0.832855,635.10,1.0


In [20]:
new_dataset['Class'].value_counts()

0.0    492
1.0    356
Name: Class, dtype: int64

In [21]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,59237.898374,-0.327173,0.072162,0.546865,0.156498,-0.293293,-0.000368,-0.112603,0.114934,0.039887,...,0.026831,-0.024657,-0.049319,-0.000593,0.029381,0.075222,0.012766,0.014944,-0.003664,90.449024
1.0,55968.873596,-6.057284,4.419181,-8.006879,4.902106,-4.402531,-1.445960,-7.057336,0.778068,-2.884994,...,0.413879,0.767732,-0.017997,-0.045050,-0.073773,0.080805,0.036841,0.181541,0.044143,112.005000


Splitting the data into Features and Targets

In [22]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [23]:
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
71582    54349.0  1.151517  0.022631  0.102991  1.134340  0.130690  0.513680   
4303      3758.0 -0.812809 -0.019916  2.123482 -1.524092 -0.078559 -0.904809   
123215   76848.0  1.335609  0.003761 -0.375240 -1.128282  0.160692 -0.505427   
11718    20031.0  0.928848  0.170297  1.276773  2.714846  0.247682  2.136157   
17849    28974.0 -0.576487  1.022091  1.628924 -0.374750  0.225452 -1.038670   
...          ...       ...       ...       ...       ...       ...       ...   
156990  109298.0 -1.000611  3.346850 -5.534491  6.835802 -0.299803  0.095951   
157585  110087.0  1.934946  0.650678 -0.286957  3.987828  0.316052 -0.099449   
157868  110547.0 -1.532810  2.232752 -5.923100  3.386708 -0.153443 -1.419748   
157871  110552.0 -2.450367  2.107729 -5.140663  1.411304 -1.690780 -0.736427   
157918  110617.0 -1.101035 -1.674928 -0.573388  5.617556  0.765556  0.440607   

              V7        V8        V9  .

In [24]:
print(Y)

71582     0.0
4303      0.0
123215    0.0
11718     0.0
17849     0.0
         ... 
156990    1.0
157585    1.0
157868    1.0
157871    1.0
157918    1.0
Name: Class, Length: 848, dtype: float64


Split the data into training data and testing data

In [25]:
X_train , X_test, Y_train , Y_test = train_test_split(X, Y, test_size=0.2, stratify = Y, random_state=2)

In [26]:
print(X.shape, X_train.shape, X_test.shape)

(848, 30) (678, 30) (170, 30)


Model Training

Logistic Regression

In [27]:
model = LogisticRegression(solver='lbfgs', max_iter=400)

In [29]:
#training the Logistic regression model with training data
model.fit(X_train, Y_train)

LogisticRegression(max_iter=400)

Model evaluation

Accuracy Score

In [30]:
# accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [34]:
print("Accuracy on train data:" ,training_data_accuracy)

Accuracy on train data: 0.9572271386430679


In [32]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [35]:
print("Accuracy on test data: ",test_data_accuracy)

Accuracy on test data:  0.9411764705882353


Random Forest Classsifier

In [38]:
rfc = RandomForestClassifier()
rfc.fit(X_train, Y_train)

RandomForestClassifier()

Accuracy Score

In [41]:
X_test_rfc = rfc.predict(X_test)
test_data_rfcaccuracy = accuracy_score(X_test_rfc, Y_test)

In [42]:
print("Accuracy on test data using Random Forest Classifier: ", test_data_rfcaccuracy)

Accuracy on test data using Random Forest Classifier:  0.9411764705882353
